# __Movielens 25m data analysis and cleaning__

#### Initial setup

In [ ]:
%load_ext watermark
%load_ext autoreload

In [ ]:
%autoreload 2
%watermark -v -n -m -p numpy,pandas,sklego

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from pandas import option_context
from sklego.pandas_utils import log_step
from collections import Counter
sys.path.append('../../../../')
from src.settings import DATA_DIR
CURRENT_PATH = os.path.abspath(os.path.join(os.pardir))
print(CURRENT_PATH)
print(DATA_DIR)

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

Import functions

In [ ]:
from src.data_processing.dataframe_utils import (col_to_onehot, start_pipeline, drop_unnecessary_cols, remove_nan, reset_index)

#### Load dataset

In [ ]:
ratings = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'ml-25m', 'ratings.csv'))
tags = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'ml-25m', 'tags.csv'))
movies = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'ml-25m', 'movies.csv'))
links = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'ml-25m', 'links.csv'))
genome_scores = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'ml-25m', 'genome-scores.csv'))
genome_tags = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'ml-25m', 'genome-tags.csv'))

### __Movielens data__

In [ ]:
def show_uniq_vals(df):
    for col in df:
        print(f'{col} - unique values: {np.unique(df[col].dropna().values).shape}')

#### __Movies__

In [ ]:
movies.head()

In [ ]:
movies[movies.genres.str.contains("IMAX")].head()

In [ ]:
movies.info()

In [ ]:
movies.isnull().values.any()

Movies genres onehot table

In [ ]:
onehot_col = 'genres'

movie_genres_onehot = (movies
                       .pipe(start_pipeline)
                       .pipe(col_to_onehot, onehot_col))

In [ ]:
 with option_context('display.max_column', None):
    display(movie_genres_onehot.head())

Changing '(no ganres listesd)' value to NaN and remove 'IMAX' from genres list

In [ ]:
from typing import Union

@log_step
def insert_nan(df: pd.DataFrame, substitute_val: str) -> pd.DataFrame:
    return df.replace(substitute_val, np.NaN)

def filter_values(string: object, drop_val: str, sep="|") -> Union[str, float]:
    return sep.join([val for val in str(string).split(sep) if val != drop_val]) if string != 'nan' else np.nan

@log_step
def remove_val_from_concat_string(df: pd.DataFrame, colname : str, value: str, sep="|") -> pd.DataFrame:  
    df[colname] = df[colname].apply(lambda x: filter_values(str(x), value, sep=sep))
    return df

In [ ]:
substitute_val = '(no genres listed)'
column = 'genres'
drop_value = 'IMAX'

movies_cleaned = (movies
                  .pipe(start_pipeline)
                  .pipe(insert_nan, substitute_val)
                  .pipe(remove_val_from_concat_string, column, drop_value))

In [ ]:
movies_cleaned.head()

In [ ]:
movies_cleaned[movies_cleaned['title'] == 'Wings of Courage (1995)']

In [ ]:
movies_cleaned[movies_cleaned.genres.str.contains("IMAX").fillna(False)].head()

In [ ]:
movies_cleaned.info()

In [ ]:
show_uniq_vals(movies_cleaned)

In [ ]:
movies_cleaned[movies_cleaned.genres.isna()].head()

#### __Ratings__

In [ ]:
ratings.head()

In [ ]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.head()

In [ ]:
ratings.info()
ratings.describe()

In [ ]:
show_uniq_vals(ratings)

In [ ]:
ratings.isnull().values.any()

In [ ]:
ratings[ratings.duplicated(subset=['userId', 'movieId'], keep=False)]

In [ ]:
ratings.movieId.unique().shape

#### __Links__

In [ ]:
links.head()

In [ ]:
links.info()

In [ ]:
show_uniq_vals(links)

In [ ]:
links_cleaned = (links
                 .pipe(start_pipeline)
                 .pipe(drop_unnecessary_cols, columns=['tmdbId'])
                 .pipe(reset_index))

In [ ]:
links_cleaned.head()

In [ ]:
links_cleaned.info()

In [ ]:
links_cleaned.isnull().any()

#### __Tags__

In [ ]:
tags.head()

In [ ]:
tags.info()

In [ ]:
tags.isnull().any()

In [ ]:
tags[tags['tag'].isnull()].head()

In [ ]:
tags_cleaned = (tags
                .pipe(start_pipeline)
                .pipe(remove_nan, columns=['tag'])
                .pipe(reset_index))

In [ ]:
tags_cleaned.info()

Tags amount describing movies

In [ ]:
show_uniq_vals(tags_cleaned)

In [ ]:
tags_cleaned.tag.unique().shape

#### __Tags genome__

In [ ]:
genome_tags.head()

In [ ]:
genome_tags.info()

In [ ]:
show_uniq_vals(genome_tags)

In [ ]:
genome_tags.isnull().any()

In [ ]:
genome_tags[genome_tags.duplicated(subset=['tag'], keep=False)]

#### __Tags genome scores__

In [ ]:
genome_scores.head()

In [ ]:
genome_scores.info()

Movies having genome scores

In [ ]:
show_uniq_vals(genome_scores)

In [ ]:
genome_scores.movieId.unique().shape

Tags included in genome scores

In [ ]:
genome_scores.tagId.unique().shape

In [ ]:
genome_scores[genome_scores.duplicated(subset=['movieId', 'tagId'])]

In [ ]:
genome_scores.isnull().any()